# Dados de Internações COVID-19

# Bibliotecas

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
#!pip install yellowbrick
from yellowbrick.classifier import ConfusionMatrix
%pylab inline


Populating the interactive namespace from numpy and matplotlib


C:\Users\Windows10\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


#  1- Leitura de Dados e Pré- Processamento



## **A-** Premissas:



Data Frame de Brasil:(**B-** df_Brasil) 
- Convertendo as variáveis com datas para datetime
- Filtro: Criação da coluna IDADE
- DATA LIMITE: 16/01/2021(Primeiro dia de vacinação no Brasil/sp)
- Remoção de duplicadas
- Remoção de variáveis não relevantes à análise (combinadas em reunião) e das que não agregam valor nenhum à Análise (nunique = 1)
- Filtro: HOSPITAL = 1 

Data Frame da cidade de São Paulo:(**C-**df_sp)
- Reset no INDEX
- Remoção de colunas sobre o cód da cidade, estado, UF.. ou seja, com unique() = 1
- Redução de categorias nas colunas 'UTI'  de 1,2,3,9 e missing foram para 1 ou 0. 
-  

## **B**- Análises dos Internados Brasil

In [4]:
df_brasil= pd.read_csv('INFLUD-13-09-2021.csv', sep= ';')
df_mcid1= df_brasil.loc[df_brasil['CO_MUN_NOT'].isin([355030, 330455, 530010, 292740, 230440, 310620, 130260, 410690,261160, 431490, 520870, 150140, 351880, 350950, 211130, 330490,270430, 330170, 240810, 500270])]
df_mcid1.head()

C:\Users\Windows10\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (15,60,62,63,64,92,94,106,108,115,117,118,119,123) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,DS_AN_OUT,TP_AM_SOR,SOR_OUT,DT_CO_SOR,TP_SOR,OUT_SOR,DT_RES,RES_IGG,RES_IGM,RES_IGA
0,10/01/2020,2,07/01/2020,2,DF,NaN,NaN,BRASILIA,530010,HOSPITAL MATERNO INFANTIL DR ANTONIO LISBOA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,07/01/2020,2,03/01/2020,1,DF,NaN,NaN,BRASILIA,530010,HRT HOSPITAL REGIONAL DE TAGUATINGA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27/01/2020,5,26/01/2020,5,CE,1 CRES FORTALEZA,1519.0,FORTALEZA,230440,HIAS HOSPITAL INFANTIL ALBERT SABIN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,12/03/2020,11,25/02/2020,9,DF,NaN,NaN,BRASILIA,530010,HRPL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,09/03/2020,11,05/03/2020,10,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,CENTRO MEDICO COMUNITARIO BAIRRO NOVO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Convertendo as variáveis com datas para datetime

In [9]:
df_mcid1['DT_NOTIFIC'] = pd.to_datetime(df_mcid1['DT_NOTIFIC'], format= "%d/%m/%Y")
df_mcid1['DT_NASC'] = pd.to_datetime(df_mcid1['DT_NASC'], format= "%d/%m/%Y")
df_mcid1['DT_EVOLUCA'] = pd.to_datetime(df_mcid1['DT_EVOLUCA'], format= "%d/%m/%Y")
df_mcid1['DT_ENCERRA'] = pd.to_datetime(df_mcid1['DT_ENCERRA'], format= "%d/%m/%Y")
df_mcid1['DT_ENTUTI'] = pd.to_datetime(df_mcid1['DT_ENCERRA'], format= "%d/%m/%Y")
df_mcid1['DT_SAIDUTI'] = pd.to_datetime(df_mcid1['DT_ENCERRA'], format= "%d/%m/%Y")
df_mcid1[['DT_NOTIFIC','DT_NASC', 'DT_EVOLUCA','DT_ENCERRA','DT_ENTUTI', 'DT_SAIDUTI']].info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 515321 entries, 0 to 1198535
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   DT_NOTIFIC  515321 non-null  datetime64[ns]
 1   DT_NASC     514158 non-null  datetime64[ns]
 2   DT_EVOLUCA  455187 non-null  datetime64[ns]
 3   DT_ENCERRA  476247 non-null  datetime64[ns]
 4   DT_ENTUTI   476247 non-null  datetime64[ns]
 5   DT_SAIDUTI  476247 non-null  datetime64[ns]
dtypes: datetime64[ns](6)
memory usage: 27.5 MB


In [10]:
df_mcid1['DT_NOTIFIC'].max()

Timestamp('2021-09-08 00:00:00')

In [11]:
df_mcid1['DT_NOTIFIC'].min()

Timestamp('2019-12-29 00:00:00')

In [12]:
df_brasil= pd.read_csv('INFLUD21-13-09-2021.csv', sep= ';')
df_mcid2= df_brasil.loc[df_brasil['CO_MUN_NOT'].isin([355030, 330455, 530010, 292740, 230440, 310620, 130260, 410690,261160, 431490, 520870, 150140, 351880, 350950, 211130, 330490,270430, 330170, 240810, 500270])]
df_mcid2.head()

C:\Users\Windows10\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (15,18,60,62,63,64,92,94,106,108,115,117,118,119,123,144) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,RES_IGM,RES_IGA,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,LAB_PR_COV,LOTE_1_COV,LOTE_2_COV,FNT_IN_COV
1,07/01/2021,1,04/01/2021,1,SP,GVE I CAPITAL,1331.0,SAO PAULO,355030,HOSP STA CRUZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,08/01/2021,1,05/01/2021,1,SP,GVE XVII CAMPINAS,1342.0,CAMPINAS,350950,HOSPITAL DAS CLINICAS DA UNICAMP DE CAMPINAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11/01/2021,2,07/01/2021,1,SP,GVE I CAPITAL,1331.0,SAO PAULO,355030,HOSP STA MAGGIORE BELA VISTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,13/01/2021,2,10/01/2021,2,SP,GVE I CAPITAL,1331.0,SAO PAULO,355030,HOSP MUN DR BENEDICTO MONTENEGRO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,11/01/2021,2,07/01/2021,1,SP,GVE XVII CAMPINAS,1342.0,CAMPINAS,350950,COMPLEXO HOSPITALAR PREFEITO EDIVALDO ORSI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Convertendo as variáveis com datas para datetime

In [13]:
df_mcid2['DT_NOTIFIC'] = pd.to_datetime(df_mcid2['DT_NOTIFIC'], format= "%d/%m/%Y")
df_mcid2['DT_NASC'] = pd.to_datetime(df_mcid2['DT_NASC'], format= "%d/%m/%Y")
df_mcid2['DT_EVOLUCA'] = pd.to_datetime(df_mcid2['DT_EVOLUCA'], format= "%d/%m/%Y")
df_mcid2['DT_ENCERRA'] = pd.to_datetime(df_mcid2['DT_ENCERRA'], format= "%d/%m/%Y")
df_mcid2['DT_ENTUTI'] = pd.to_datetime(df_mcid2['DT_ENCERRA'], format= "%d/%m/%Y")
df_mcid2['DT_SAIDUTI'] = pd.to_datetime(df_mcid2['DT_ENCERRA'], format= "%d/%m/%Y")
df_mcid2[['DT_NOTIFIC','DT_NASC', 'DT_EVOLUCA','DT_ENCERRA','DT_ENTUTI', 'DT_SAIDUTI']].info() 

C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_39308/3273959113.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mcid2['DT_NOTIFIC'] = pd.to_datetime(df_mcid2['DT_NOTIFIC'], format= "%d/%m/%Y")
C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_39308/3273959113.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mcid2['DT_NASC'] = pd.to_datetime(df_mcid2['DT_NASC'], format= "%d/%m/%Y")
C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_39308/3273959113.py:3: SettingWithCopyWarning: 
A value is trying to be

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499196 entries, 1 to 1471563
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   DT_NOTIFIC  499196 non-null  datetime64[ns]
 1   DT_NASC     498879 non-null  datetime64[ns]
 2   DT_EVOLUCA  395730 non-null  datetime64[ns]
 3   DT_ENCERRA  418620 non-null  datetime64[ns]
 4   DT_ENTUTI   418620 non-null  datetime64[ns]
 5   DT_SAIDUTI  418620 non-null  datetime64[ns]
dtypes: datetime64[ns](6)
memory usage: 26.7 MB


C:\Users\WINDOW~1\AppData\Local\Temp/ipykernel_39308/3273959113.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mcid2['DT_SAIDUTI'] = pd.to_datetime(df_mcid2['DT_ENCERRA'], format= "%d/%m/%Y")


In [14]:
df_mcid2['DT_NOTIFIC'].max()

Timestamp('2021-09-13 00:00:00')

In [15]:
df_mcid2['DT_NOTIFIC'].min()

Timestamp('2021-01-03 00:00:00')

- DATA LIMITE: 16/01/2021  
df_mcid1 = Base antes da Vacina
df_mcid2 = Base durante a Vacina

In [16]:
df_mcid1=df_mcid1.loc[df_mcid1['DT_NOTIFIC']<='2021-01-16']

In [17]:
df_mcid2=df_mcid2.loc[df_mcid2['DT_NOTIFIC']>'2021-01-16']

In [18]:
df_mcid=pd.concat([df_mcid1, df_mcid2])
df_mcid

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,RES_IGM,RES_IGA,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,LAB_PR_COV,LOTE_1_COV,LOTE_2_COV,FNT_IN_COV
0,2020-01-10,2,07/01/2020,2,DF,NaN,NaN,BRASILIA,530010,HOSPITAL MATERNO INFANTIL DR ANTONIO LISBOA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-07,2,03/01/2020,1,DF,NaN,NaN,BRASILIA,530010,HRT HOSPITAL REGIONAL DE TAGUATINGA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-27,5,26/01/2020,5,CE,1 CRES FORTALEZA,1519.0,FORTALEZA,230440,HIAS HOSPITAL INFANTIL ALBERT SABIN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-03-12,11,25/02/2020,9,DF,NaN,NaN,BRASILIA,530010,HRPL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020-03-09,11,05/03/2020,10,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,CENTRO MEDICO COMUNITARIO BAIRRO NOVO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471551,2021-09-10,36,30/08/2021,35,SP,GVE I CAPITAL,1331.0,SAO PAULO,355030,HOSPITAL DO SERV PUB EST FCO MORATO DE OLIVEIR...,...,NaN,NaN,2.0,1.0,NaN,NaN,ASTRAZENICA 2 DOSES,NaN,NaN,1.0
1471555,2021-09-02,35,29/08/2021,35,SP,GVE I CAPITAL,1331.0,SAO PAULO,355030,HOSPITAL DO SERV PUB EST FCO MORATO DE OLIVEIR...,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1471559,2021-08-30,35,22/08/2021,34,RJ,NaN,NaN,RIO DE JANEIRO,330455,HOSPITAL QUINTA DOR,...,NaN,NaN,2.0,1.0,23/03/2021,20/04/2021,SINOVAC,NaN,NaN,1.0
1471561,2021-07-24,29,24/07/2021,29,RJ,NaN,NaN,DUQUE DE CAXIAS,330170,SES RJ UPA 24H SARAPUI,...,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,1.0


- Variáveis : Combinamos de manter as variáveis que tenham pelo menos um SIM [nesta tabela](https://docs.google.com/spreadsheets/d/17-jccz8DWl_fW7NpA0d6A3wQ3Ntc8qKX/edit#gid=1202247063) a partir da interpretação do Dicionário.

OBS: Nesta base atualizada existem 8 variáveis a mais do que a antiga:

'ESTRANG', 'VACINA_COV', 'DOSE_1_COV', 'DOSE_2_COV', 'LAB_PR_COV', 'LOTE_1_COV', 'LOTE_2_COV', 'FNT_IN_COV'.

Essas colunas também foram dropadas.




In [22]:
df_mcid.drop(columns=[ 'SEM_NOT','DT_SIN_PRI','SEM_PRI','ID_REGIONA','CO_REGIONA', 'ID_UNIDADE', 'CO_UNI_NOT','ID_PAIS','CO_PAIS',
                      'SG_UF', 'ID_RG_RESI','CO_RG_RESI','ID_MN_RESI','CO_MUN_RES','CS_ZONA',
                      'CS_ETINIA', 'PAC_COCBO','PAC_DSCBO','PAIS_VGM','CO_PS_VGM','LO_PS_VGM','DT_VGM','DT_RT_VGM',
                      'OUTRO_DES','MORB_DESC','DT_UT_DOSE','MAE_VAC', 'DT_VAC_MAE',
                      'M_AMAMENTA','DT_DOSEUNI', 'DT_1_DOSE','DT_2_DOSE','TP_ANTIVIR','OUT_ANTIV','DT_ANTIVIR','DT_INTERNA',
                      'ID_RG_INTE','CO_RG_INTE', 'ID_MN_INTE','RAIOX_OUT','DT_RAIOX','TOMO_OUT','DT_TOMO',
                      'AMOSTRA','DT_COLETA','TP_AMOSTRA','OUT_AMOST', 'DT_PCR','POS_PCRFLU','TP_FLU_PCR','PCR_FLUASU','FLUASU_OUT',
                      'PCR_FLUBLI','FLUBLI_OUT','POS_PCROUT','PCR_SARS2', 'PCR_VSR','PCR_PARA1','PCR_PARA2','PCR_PARA3','PCR_PARA4',
                      'PCR_ADENO','PCR_METAP','PCR_BOCA','PCR_RINO','PCR_OUTRO', 'DS_PCR_OUT','CLASSI_FIN','CLASSI_OUT','CRITERIO',
                      'TP_TES_AN','DT_RES_AN','RES_AN','POS_AN_FLU','TP_FLU_AN','POS_AN_OUT', 'AN_SARS2','AN_VSR','AN_PARA1',
                      'AN_PARA2','AN_PARA3','AN_ADENO','AN_OUTRO','DS_AN_OUT','TP_AM_SOR','SOR_OUT','DT_CO_SOR', 'TP_SOR','OUT_SOR',
                      'DT_RES','DT_DIGITA','OBES_IMC', 'OUT_ANIM', 'ESTRANG', 'VACINA_COV', 'DOSE_1_COV', 'DOSE_2_COV', 'LAB_PR_COV', 
                      'LOTE_1_COV', 'LOTE_2_COV', 'FNT_IN_COV'], inplace=True)

In [23]:

idade_aux = []

for i,j in enumerate (df_mcid.TP_IDADE):
  if j == 2:
    idade_aux.append(df_mcid.NU_IDADE_N[i]/12)
  elif j == 1:
    idade_aux.append(df_mcid.NU_IDADE_N[i]/360)
  else:
    idade_aux.append(df_mcid.NU_IDADE_N[i])
    
df_mcid["IDADE_ANOS"] = idade_aux


KeyError: 2

In [ ]:
df_mcid[["IDADE_ANOS",'NU_IDADE_N','TP_IDADE','COD_IDADE']]

- Filtro: Criação da coluna IDADE e drop nas outras . 

OBS: No campo de 'NU_IDADE_N','TP_IDADE','COD_IDADE',devemos dar atenção maior se as idades forem menores que 1 ano:

Ex: Paciente apresenta, na coluna da idade, o valor 8, porém na coluna seguinte (que classifica em 1:dias, 2:meses, 3: anos) apresenta o valor 2 para este paciente. Então não são 8 anos e sim, 8 meses.( 0,67 anos) 

Vamos então criar uma coluna com idade em float e não int.



In [ ]:
df_brasil.drop(columns=['NU_IDADE_N','TP_IDADE','COD_IDADE','DT_NASC'], inplace=True)
df_brasil.head()


In [ ]:
df_brasil_AV['DT_NOTIFIC'].min()

In [ ]:
df_brasil_AV['DT_NOTIFIC'].max()

- Linhas duplicadas


In [ ]:
print(f'Temos {df_brasil_AV.duplicated().sum()} linhas duplicadas.')

In [ ]:
df_brasil_AV=df_brasil_AV.drop_duplicates()

Foi decidido pelo grupo dropar as 10 linhas duplicadas.



- Tamanho df_brasil



In [ ]:
df_brasil_AV.shape

In [ ]:
df_brasil_AV.shape

De 162 fomos para 61 colunas. 

- Informações df_brasil_AV

In [ ]:
df_brasil_AV.info()

- Missing

In [ ]:
df_brasil_AV.isna().sum()

In [ ]:
# Porcentagem de missing
df_brasil_AV.isnull().sum()/len(df_brasil_AV)

- Avaliação de Variáveis com unique()= 1

In [ ]:
# Analise das colunas que provavelmente vamos dropar:
print(list(df_brasil_AV.nunique()))
#As que estão com 1 são constantes em todo df_sp.

- Filtro HOSPITAL = 1 :

Significa que o paciente foi internado. No dicionário 2 = Não, 9 = Ignorado e missing (2,55%) vamos assumir como também não internado.


In [ ]:
# Porcentagem de missing
df_brasil_AV['HOSPITAL'].isnull().sum()/len(df_brasil_AV)


In [ ]:
df_brasil_AV= df_brasil_AV[(df_brasil_AV['HOSPITAL']==1.0)]
df_brasil_AV['HOSPITAL'].nunique()

In [ ]:
df_brasil_AV['HOSPITAL'].isnull().sum()/len(df_brasil_AV)

In [ ]:
df_brasil_AV.shape

## **C-** Análise dos Internados na cidade de São Paulo.

- Filtro: cidade de SP. 

In [ ]:
df_sp= df_brasil_AV.loc[df_brasil_AV['CO_MUN_NOT'].isin([355030])]

In [ ]:
df_sp.shape

In [ ]:
# porcentagem da cidade de São Paulo frente ao BR: 12,35%
len(df_sp)/len(df_brasil_AV)

- Reset no INDEX  por conta do filtro

In [ ]:
df_sp= df_sp.reset_index(drop=True)
df_sp.head()

- Remoção de variáveis com valores constantes

In [ ]:
print(list(df_sp.nunique()))

In [ ]:
#Dropando as colunas:
df_sp.drop(columns=[ 'SG_UF_NOT','ID_MUNICIP','CO_MUN_NOT','HOSPITAL'], inplace=True)
df_sp.shape

In [ ]:
df_sp.nunique()

- Observação dos Missing values

In [ ]:
# Porcentagem de missing
df_sp.isnull().sum()/len(df_sp)

Combinamos que muitos Nulls são por falta de preenchimento do campo. Então se não foi preenchido alguma comorbidade ou sintoma, por exemplo, seria assumido que a pessoa não possui. 

###- Análise de cada variável / Possível Replace para algumas (1 e 0) 


In [ ]:
#ex: UTI  " O paciente foi internado em UTI?"
# 1- Sim; 2- Não ou 9-Ignorado.
#Assumimos que 2, 9 e Missing( 9 %) representam uma resposta de que não foi a UTI, assim foram substituidos por 0.

In [ ]:
df_sp['CS_SEXO'].nunique()
print(df_sp['CS_SEXO'].unique())
#1 - Masculino, 2- Feminino. 

In [ ]:
df_sp['CS_SEXO'].replace({"F": 0, "M": 1}, inplace= True)
print(df_sp['CS_SEXO'].unique())
df_sp['CS_SEXO'].isnull().sum()/len(df_sp['CS_SEXO'])
#NOVO DICIONÁRIO: 1- Masculino, 0- Feminino

In [ ]:
print(df_sp['CS_GESTANT'].nunique())
print(df_sp['CS_GESTANT'].unique())
df_sp['CS_GESTANT'].isnull().sum()/len(df_sp['CS_GESTANT'])
#1-1o Trimestre,2-2o Trimestre, 3-3o Trimestre, 4-Idade Gestacional Ignorada, 5-Não, 6-Não se aplica, 9-Ignorado.

In [ ]:
df_sp['CS_GESTANT'].replace({1.0: 1, 2.0: 1, 3.0  :1, 4.0 : 1}, inplace= True)
df_sp['CS_GESTANT'].replace({5.0: 0, 6.0:0, 9.0:0}, inplace= True)

 #NOVO DICIONÁRIO: Gestante:1 = 1,2,3. Não Gestante: 0 = Missing,5,6,9
print(df_sp['CS_GESTANT'].unique())

In [ ]:
print(df_sp['CS_RACA'].nunique())
print(df_sp['CS_RACA'].unique())
df_sp['CS_RACA'].isnull().sum()/len(df_sp['CS_RACA'])
#1-Branca,2-Preta,3-Amarela,4-Parda,5-Indígena,9-Ignorado

In [ ]:
df_sp['CS_RACA'].fillna(9,inplace= True)
print(df_sp['CS_RACA'].unique())

In [ ]:
print(df_sp['CS_ESCOL_N'].nunique())
print(df_sp['CS_ESCOL_N'].unique())
df_sp['CS_ESCOL_N'].isnull().sum()/len(df_sp['CS_ESCOL_N'])
#0-Sem escolaridade/Analfabeto,1-Fundamental 1o ciclo (1a a 5a série),2-Fundamental 2o ciclo (6a a 9a série),3- Médio (1o ao 3o ano),
#4-Superior,5-Não se aplica(menor que 7 anos),9-Ignorado

In [ ]:
df_sp['CS_ESCOL_N'].fillna(9,inplace= True)
print(df_sp['CS_ESCOL_N'].unique())


In [ ]:
print(df_sp['SURTO_SG'].nunique())
print(df_sp['SURTO_SG'].unique())
df_sp['SURTO_SG'].isnull().sum()/len(df_sp['SURTO_SG'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['SURTO_SG'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['SURTO_SG'].fillna(0,inplace= True)
print(df_sp['SURTO_SG'].unique())
df_sp['SURTO_SG'].isnull().sum()/len(df_sp['SURTO_SG'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['NOSOCOMIAL'].nunique())
print(df_sp['NOSOCOMIAL'].unique())
df_sp['NOSOCOMIAL'].isnull().sum()/len(df_sp['NOSOCOMIAL'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['NOSOCOMIAL'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['NOSOCOMIAL'].fillna(0,inplace= True)
print(df_sp['NOSOCOMIAL'].unique())
df_sp['NOSOCOMIAL'].isnull().sum()/len(df_sp['NOSOCOMIAL'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['AVE_SUINO'].nunique())
print(df_sp['AVE_SUINO'].unique())
df_sp['AVE_SUINO'].isnull().sum()/len(df_sp['AVE_SUINO'])
#1-Sim, 2-Não,9-Ignorado


In [ ]:
## Pelo dicionário não deveria ter o 3, acredito que tenha sido um erro 
df_ave = df_sp.loc[df_sp['AVE_SUINO']==3]
df_ave.shape 
# 15 linhas, vou considerar como erro. 

In [ ]:
df_sp['AVE_SUINO'].replace({2.0: 0, 9.0: 0, 3.0: 0}, inplace= True)
df_sp['AVE_SUINO'].fillna(0,inplace= True)
print(df_sp['AVE_SUINO'].unique())
df_sp['AVE_SUINO'].isnull().sum()/len(df_sp['AVE_SUINO'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['FEBRE'].nunique())
print(df_sp['FEBRE'].unique())
df_sp['FEBRE'].isnull().sum()/len(df_sp['FEBRE'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['FEBRE'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['FEBRE'].fillna(0,inplace= True)
print(df_sp['FEBRE'].unique())
df_sp['FEBRE'].isnull().sum()/len(df_sp['FEBRE'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['TOSSE'].nunique())
print(df_sp['TOSSE'].unique())
df_sp['TOSSE'].isnull().sum()/len(df_sp['TOSSE'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['TOSSE'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['TOSSE'].fillna(0,inplace= True)
print(df_sp['TOSSE'].unique())
df_sp['TOSSE'].isnull().sum()/len(df_sp['TOSSE'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['GARGANTA'].nunique())
print(df_sp['GARGANTA'].unique())
df_sp['GARGANTA'].isnull().sum()/len(df_sp['GARGANTA'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['GARGANTA'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['GARGANTA'].fillna(0,inplace= True)
print(df_sp['GARGANTA'].unique())
df_sp['GARGANTA'].isnull().sum()/len(df_sp['GARGANTA'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['DISPNEIA'].nunique())
print(df_sp['DISPNEIA'].unique())
df_sp['DISPNEIA'].isnull().sum()/len(df_sp['DISPNEIA'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['DISPNEIA'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['DISPNEIA'].fillna(0,inplace= True)
print(df_sp['DISPNEIA'].unique())
df_sp['DISPNEIA'].isnull().sum()/len(df_sp['DISPNEIA'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['DESC_RESP'].nunique())
print(df_sp['DESC_RESP'].unique())
df_sp['DESC_RESP'].isnull().sum()/len(df_sp['DESC_RESP'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['DESC_RESP'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['DESC_RESP'].fillna(0,inplace= True)
print(df_sp['DESC_RESP'].unique())
df_sp['DESC_RESP'].isnull().sum()/len(df_sp['DESC_RESP'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['SATURACAO'].nunique())
print(df_sp['SATURACAO'].unique())
df_sp['SATURACAO'].isnull().sum()/len(df_sp['SATURACAO'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['SATURACAO'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['SATURACAO'].fillna(0,inplace= True)
print(df_sp['SATURACAO'].unique())
df_sp['SATURACAO'].isnull().sum()/len(df_sp['SATURACAO'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['DIARREIA'].nunique())
print(df_sp['DIARREIA'].unique())
df_sp['DIARREIA'].isnull().sum()/len(df_sp['DIARREIA'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['DIARREIA'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['DIARREIA'].fillna(0,inplace= True)
print(df_sp['DIARREIA'].unique())
df_sp['DIARREIA'].isnull().sum()/len(df_sp['DIARREIA'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['VOMITO'].nunique())
print(df_sp['VOMITO'].unique())
df_sp['VOMITO'].isnull().sum()/len(df_sp['VOMITO'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['VOMITO'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['VOMITO'].fillna(0,inplace= True)
print(df_sp['VOMITO'].unique())
df_sp['VOMITO'].isnull().sum()/len(df_sp['VOMITO'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['OUTRO_SIN'].nunique())
print(df_sp['OUTRO_SIN'].unique())
df_sp['OUTRO_SIN'].isnull().sum()/len(df_sp['OUTRO_SIN'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['OUTRO_SIN'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['OUTRO_SIN'].fillna(0,inplace= True)
print(df_sp['OUTRO_SIN'].unique())
df_sp['OUTRO_SIN'].isnull().sum()/len(df_sp['OUTRO_SIN'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['PUERPERA'].nunique())
print(df_sp['PUERPERA'].unique())
df_sp['PUERPERA'].isnull().sum()/len(df_sp['PUERPERA'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['PUERPERA'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['PUERPERA'].fillna(0,inplace= True)
print(df_sp['PUERPERA'].unique())
df_sp['PUERPERA'].isnull().sum()/len(df_sp['PUERPERA'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['FATOR_RISC'].nunique())
print(df_sp['FATOR_RISC'].unique())
df_sp['FATOR_RISC'].isnull().sum()/len(df_sp['FATOR_RISC'])
#S-Sim, N-Não

In [ ]:
#df_sp['FATOR_RISC'].replace({'S': 1, 'N': 0}, inplace= True)
#print(df_sp['FATOR_RISC'].unique())
#df_sp['FATOR_RISC'].isnull().sum()/len(df_sp[''])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['CARDIOPATI'].nunique())
print(df_sp['CARDIOPATI'].unique())
df_sp['CARDIOPATI'].isnull().sum()/len(df_sp['CARDIOPATI'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['CARDIOPATI'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['CARDIOPATI'].fillna(0,inplace= True)
print(df_sp['CARDIOPATI'].unique())
df_sp['CARDIOPATI'].isnull().sum()/len(df_sp['CARDIOPATI'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['HEMATOLOGI'].nunique())
print(df_sp['HEMATOLOGI'].unique())
df_sp['HEMATOLOGI'].isnull().sum()/len(df_sp['HEMATOLOGI'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['HEMATOLOGI'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['HEMATOLOGI'].fillna(0,inplace= True)
print(df_sp['HEMATOLOGI'].unique())
df_sp['HEMATOLOGI'].isnull().sum()/len(df_sp['HEMATOLOGI'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['SIND_DOWN'].nunique())
print(df_sp['SIND_DOWN'].unique())
df_sp['SIND_DOWN'].isnull().sum()/len(df_sp['SIND_DOWN'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['SIND_DOWN'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['SIND_DOWN'].fillna(0,inplace= True)
print(df_sp['SIND_DOWN'].unique())
df_sp['SIND_DOWN'].isnull().sum()/len(df_sp['SIND_DOWN'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['HEPATICA'].nunique())
print(df_sp['HEPATICA'].unique())
df_sp['HEPATICA'].isnull().sum()/len(df_sp['HEPATICA'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['HEPATICA'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['HEPATICA'].fillna(0,inplace= True)
print(df_sp['HEPATICA'].unique())
df_sp['HEPATICA'].isnull().sum()/len(df_sp['HEPATICA'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['ASMA'].nunique())
print(df_sp['ASMA'].unique())
df_sp['ASMA'].isnull().sum()/len(df_sp['ASMA'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['ASMA'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['ASMA'].fillna(0,inplace= True)
print(df_sp['ASMA'].unique())
df_sp['ASMA'].isnull().sum()/len(df_sp['ASMA'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['DIABETES'].nunique())
print(df_sp['DIABETES'].unique())
df_sp['DIABETES'].isnull().sum()/len(df_sp['DIABETES'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['DIABETES'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['DIABETES'].fillna(0,inplace= True)
print(df_sp['DIABETES'].unique())
df_sp['DIABETES'].isnull().sum()/len(df_sp['DIABETES'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['NEUROLOGIC'].nunique())
print(df_sp['NEUROLOGIC'].unique())
df_sp['NEUROLOGIC'].isnull().sum()/len(df_sp['NEUROLOGIC'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['NEUROLOGIC'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['NEUROLOGIC'].fillna(0,inplace= True)
print(df_sp['NEUROLOGIC'].unique())
df_sp['NEUROLOGIC'].isnull().sum()/len(df_sp['NEUROLOGIC'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['PNEUMOPATI'].nunique())
print(df_sp['PNEUMOPATI'].unique())
df_sp['PNEUMOPATI'].isnull().sum()/len(df_sp['PNEUMOPATI'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['PNEUMOPATI'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['PNEUMOPATI'].fillna(0,inplace= True)
print(df_sp['PNEUMOPATI'].unique())
df_sp['PNEUMOPATI'].isnull().sum()/len(df_sp['PNEUMOPATI'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['IMUNODEPRE'].nunique())
print(df_sp['IMUNODEPRE'].unique())
df_sp['IMUNODEPRE'].isnull().sum()/len(df_sp['IMUNODEPRE'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['IMUNODEPRE'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['IMUNODEPRE'].fillna(0,inplace= True)
print(df_sp['IMUNODEPRE'].unique())
df_sp['IMUNODEPRE'].isnull().sum()/len(df_sp['IMUNODEPRE'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['RENAL'].nunique())
print(df_sp['RENAL'].unique())
df_sp['RENAL'].isnull().sum()/len(df_sp['RENAL'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['RENAL'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['RENAL'].fillna(0,inplace= True)
print(df_sp['RENAL'].unique())
df_sp['RENAL'].isnull().sum()/len(df_sp['RENAL'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['OBESIDADE'].nunique())
print(df_sp['OBESIDADE'].unique())
df_sp['OBESIDADE'].isnull().sum()/len(df_sp['OBESIDADE'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['OBESIDADE'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['OBESIDADE'].fillna(0,inplace= True)
print(df_sp['OBESIDADE'].unique())
df_sp['OBESIDADE'].isnull().sum()/len(df_sp['OBESIDADE'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['OUT_MORBI'].nunique())
print(df_sp['OUT_MORBI'].unique())
df_sp['OUT_MORBI'].isnull().sum()/len(df_sp['OUT_MORBI'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['OUT_MORBI'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['OUT_MORBI'].fillna(0,inplace= True)
print(df_sp['OUT_MORBI'].unique())
df_sp['OUT_MORBI'].isnull().sum()/len(df_sp['OUT_MORBI'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['VACINA'].nunique())
print(df_sp['VACINA'].unique())
df_sp['VACINA'].isnull().sum()/len(df_sp['VACINA'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['ASMA'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['ASMA'].fillna(0,inplace= True)
print(df_sp['ASMA'].unique())
df_sp['ASMA'].isnull().sum()/len(df_sp['ASMA'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['ANTIVIRAL'].nunique())
print(df_sp['ANTIVIRAL'].unique())
df_sp['ANTIVIRAL'].isnull().sum()/len(df_sp['ANTIVIRAL'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['ANTIVIRAL'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['ANTIVIRAL'].fillna(0,inplace= True)
print(df_sp['ANTIVIRAL'].unique())
df_sp['ANTIVIRAL'].isnull().sum()/len(df_sp['ANTIVIRAL'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['SG_UF_INTE'].nunique())
print(df_sp['SG_UF_INTE'].unique())
df_sp['SG_UF_INTE'].isnull().sum()/len(df_sp['SG_UF_INTE'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_uf = df_sp.loc[df_sp['SG_UF_INTE']=='MA']
df_uf.shape
#1 linha apenas... Vou dropar essa coluna então pq ela não agrega info significativa ao à análise.

In [ ]:
df_sp.drop(columns=['SG_UF_INTE'], inplace=True)
# df_sp['SG_UF_INTE'] # Não entendi porque visualizar depois de dropar...

In [ ]:
df_sp.shape

In [ ]:
print(df_sp['UTI'].nunique())
print(df_sp['UTI'].unique())
df_sp['UTI'].isnull().sum()/len(df_sp['UTI'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp['UTI'].replace({2.0: 0, 9.0: 0}, inplace= True)
df_sp['UTI'].fillna(0,inplace= True)
print(df_sp['UTI'].unique())
df_sp['UTI'].isnull().sum()/len(df_sp['UTI'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não

In [ ]:
print(df_sp['DT_ENTUTI'].isnull().sum()/len(df_sp['DT_ENTUTI'])) # 7,3% missing para as duas variáveis
print(df_sp['DT_SAIDUTI'].isnull().sum()/len(df_sp['DT_SAIDUTI'])) 

In [ ]:
print(df_sp['SUPORT_VEN'].nunique())
print(df_sp['SUPORT_VEN'].unique())
df_sp['SUPORT_VEN'].isnull().sum()/len(df_sp['SUPORT_VEN'])
#1-Sim, invasivo 2-Sim, não invasivo 3-Não 9-Ignorado

In [ ]:
df_sp['SUPORT_VEN'].replace({3.0: 0, 9.0: 0}, inplace= True)
df_sp['SUPORT_VEN'].fillna(0,inplace= True)
print(df_sp['SUPORT_VEN'].unique())
df_sp['SUPORT_VEN'].isnull().sum()/len(df_sp['SUPORT_VEN'])
#NOVO DICIONÁRIO: 1- 1-Sim, invasivo,  2-Sim, não invasivo,  0- Não

In [ ]:
print(df_sp['RAIOX_RES'].nunique())
print(df_sp['RAIOX_RES'].unique())
df_sp['RAIOX_RES'].isnull().sum()/len(df_sp['RAIOX_RES'])
#1-Normal
#2-Infiltrado intersticial
#3-Consolidação
#4-Misto
#5-Outro
#6-Não realizado
#9-Ignorado

In [ ]:
df_sp['RAIOX_RES'].replace({ 9.0: 6}, inplace= True)
df_sp['RAIOX_RES'].fillna(6,inplace= True)
print(df_sp['RAIOX_RES'].unique())
df_sp['RAIOX_RES'].isnull().sum()/len(df_sp['RAIOX_RES'])
#NOVO DICIONÁRIO: 1- Sim, 0- Não
#1-Normal
#2-Infiltrado intersticial
#3-Consolidação
#4-Misto
#5-Outro
#6-Não realizado

In [ ]:
print(df_sp['PCR_RESUL'].nunique())
print(df_sp['PCR_RESUL'].unique())
df_sp['PCR_RESUL'].isnull().sum()/len(df_sp['PCR_RESUL'])
#1-Detectável
#2-Não Detectável
#3-Inconclusivo
#4-Não Realizado
#5-Aguardando Resultado
#9-Ignorado

In [ ]:
df_sp['PCR_RESUL'].fillna(4,inplace= True)
print(df_sp['PCR_RESUL'].unique())
df_sp['PCR_RESUL'].isnull().sum()/len(df_sp['PCR_RESUL'])


- Filtro : EVOLUÇÃO = 1 e 2.

In [ ]:
print(df_sp['EVOLUCAO'].nunique())
print(df_sp['EVOLUCAO'].unique())
df_sp['EVOLUCAO'].isnull().sum()/len(df_sp['EVOLUCAO'])
#1-Cura
#2-Óbito
#3-Óbito por outras causas
#9-Ignorado

In [ ]:
df_evo1 = df_sp.loc[df_sp['EVOLUCAO']==1]
len(df_evo1)/len(df_sp)

In [ ]:
df_evo2 = df_sp.loc[df_sp['EVOLUCAO']==2]
len(df_evo2)/len(df_sp)

In [ ]:
df_evo3 = df_sp.loc[df_sp['EVOLUCAO']==3]
len(df_evo3)/len(df_sp)

In [ ]:
df_evo9 = df_sp.loc[df_sp['EVOLUCAO']==9]
len(df_evo9)/len(df_sp)

In [ ]:
df_sp['EVOLUCAO'].isnull().sum()/len(df_sp)


In [ ]:
df_sp_1= df_sp.loc[(df_sp['EVOLUCAO']<=2.0)]
print(df_sp_1['EVOLUCAO'].unique())

In [ ]:
df_sp_1.head()

In [ ]:
df_sp_1.shape

- Resetando o Index novamente.

In [ ]:
df_sp_1= df_sp_1.reset_index(drop=True)
df_sp_1.head()

- Continuando a análise das variáveis 


In [ ]:
print(df_sp_1['DT_EVOLUCA'].isnull().sum()/len(df_sp['DT_EVOLUCA'])) # 8,4% missing 
print(df_sp_1['DT_ENCERRA'].isnull().sum()/len(df_sp['DT_ENCERRA'])) # 7,2% missing 

In [ ]:
print(df_sp_1['HISTO_VGM'].nunique())
print(df_sp_1['HISTO_VGM'].unique())
df_sp_1['HISTO_VGM'].isnull().sum()/len(df_sp_1['HISTO_VGM'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
#Deve ter algum erro pq no dicionario não tem 0 
df_sp_1['HISTO_VGM'].replace({0: 2}, inplace= True)
print(df_sp_1['HISTO_VGM'].unique())
df_sp_1['HISTO_VGM'].isnull().sum()/len(df_sp_1['HISTO_VGM'])
#NOVO DICIONÁRIO: #1-Sim, 2-Não,9-Ignorado

In [ ]:
print(df_sp_1['DOR_ABD'].nunique())
print(df_sp_1['DOR_ABD'].unique())
df_sp_1['DOR_ABD'].isnull().sum()/len(df_sp_1['DOR_ABD'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp_1['DOR_ABD'].replace({9.0: 0, 2.0 :0}, inplace= True)
df_sp_1['DOR_ABD'].fillna(0,inplace= True)
print(df_sp_1['DOR_ABD'].unique())
df_sp_1['DOR_ABD'].isnull().sum()/len(df_sp_1['DOR_ABD'])
#NOVO DICIONÁRIO: #1-Sim, 0-Não


In [ ]:
print(df_sp_1['FADIGA'].nunique())
print(df_sp_1['FADIGA'].unique())
df_sp_1['FADIGA'].isnull().sum()/len(df_sp_1['FADIGA'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp_1['FADIGA'].replace({9.0: 0, 2.0 :0}, inplace= True)
df_sp_1['FADIGA'].fillna(0,inplace= True)
print(df_sp_1['FADIGA'].unique())
df_sp_1['FADIGA'].isnull().sum()/len(df_sp_1['FADIGA'])
#NOVO DICIONÁRIO: #1-Sim, 0-Não


In [ ]:
print(df_sp_1['PERD_OLFT'].nunique())
print(df_sp_1['PERD_OLFT'].unique())
df_sp_1['PERD_OLFT'].isnull().sum()/len(df_sp_1['PERD_OLFT'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp_1['PERD_OLFT'].replace({9.0: 0, 2.0 :0}, inplace= True)
df_sp_1['PERD_OLFT'].fillna(0,inplace= True)
print(df_sp_1['PERD_OLFT'].unique())
df_sp_1['PERD_OLFT'].isnull().sum()/len(df_sp_1['PERD_OLFT'])
#NOVO DICIONÁRIO: #1-Sim, 0-Não


In [ ]:
print(df_sp_1['PERD_PALA'].nunique())
print(df_sp_1['PERD_PALA'].unique())
df_sp_1['PERD_PALA'].isnull().sum()/len(df_sp_1['PERD_PALA'])
#1-Sim, 2-Não,9-Ignorado

In [ ]:
df_sp_1['PERD_PALA'].replace({9.0: 0, 2.0 :0}, inplace= True)
df_sp_1['PERD_PALA'].fillna(0,inplace= True)
print(df_sp_1['PERD_PALA'].unique())
df_sp_1['PERD_PALA'].isnull().sum()/len(df_sp_1['PERD_PALA'])
#NOVO DICIONÁRIO: #1-Sim, 0-Não


In [ ]:
print(df_sp_1['TOMO_RES'].nunique())
print(df_sp_1['TOMO_RES'].unique())
df_sp_1['TOMO_RES'].isnull().sum()/len(df_sp_1['TOMO_RES'])
#1-Tipico COVID-19
#2- Indeterminado COVID-19
#3- Atípico COVID-19
#4- Negativo para Pneumonia
#5- Outro
#6-Não realizado
#9-Ignorado

In [ ]:
df_sp_1['TOMO_RES'].replace({9.0: 6}, inplace= True)
df_sp_1['TOMO_RES'].fillna(6,inplace= True)
print(df_sp_1['TOMO_RES'].unique())
df_sp_1['TOMO_RES'].isnull().sum()/len(df_sp_1['PERD_OLFT'])
#NOVO DICIONÁRIO: 
#1-Tipico COVID-19
#2- Indeterminado COVID-19
#3- Atípico COVID-19
#4- Negativo para Pneumonia
#5- Outro
#6-Não realizado
#9-Ignorado

In [ ]:
print(df_sp_1['RES_IGG'].nunique())
print(df_sp_1['RES_IGG'].unique())
df_sp_1['RES_IGG'].isnull().sum()/len(df_sp_1['RES_IGG'])
#1-Positivo
#2-Negativo
#3-Inconclusivo
#4-Não realizado
#5-Aguarda resultado
#9 Ignorado

In [ ]:
df_sp_1['RES_IGG'].replace({9.0: 4}, inplace= True)
df_sp_1['RES_IGG'].fillna(4,inplace= True)
print(df_sp_1['RES_IGG'].unique())
df_sp_1['RES_IGG'].isnull().sum()/len(df_sp_1['RES_IGG'])
#NOVO DICIONÁRIO: 
#1-Positivo
#2-Negativo
#3-Inconclusivo
#4-Não realizado
#5-Aguarda resultado

In [ ]:
print(df_sp_1['RES_IGM'].nunique())
print(df_sp_1['RES_IGM'].unique())
df_sp_1['RES_IGM'].isnull().sum()/len(df_sp_1['RES_IGM'])
#1-Positivo
#2-Negativo
#3-Inconclusivo
#4-Não realizado
#5-Aguarda resultado
#9 Ignorado

In [ ]:
df_sp_1['RES_IGM'].replace({9.0: 4}, inplace= True)
df_sp_1['RES_IGM'].fillna(4,inplace= True)
print(df_sp_1['RES_IGM'].unique())
df_sp_1['RES_IGM'].isnull().sum()/len(df_sp_1['RES_IGM'])
#NOVO DICIONÁRIO: 
#1-Positivo
#2-Negativo
#3-Inconclusivo
#4-Não realizado
#5-Aguarda resultado

In [ ]:
print(df_sp_1['RES_IGA'].nunique())
print(df_sp_1['RES_IGA'].unique())
df_sp_1['RES_IGA'].isnull().sum()/len(df_sp_1['RES_IGA'])
#1-Positivo
#2-Negativo
#3-Inconclusivo
#4-Não realizado
#5-Aguarda resultado
#9 Ignorado

In [ ]:
df_sp_1['RES_IGA'].replace({9.0: 4}, inplace= True)
df_sp_1['RES_IGA'].fillna(4,inplace= True)
print(df_sp_1['RES_IGA'].unique())
df_sp_1['RES_IGA'].isnull().sum()/len(df_sp_1['RES_IGA'])
#NOVO DICIONÁRIO: 
#1-Positivo
#2-Negativo
#3-Inconclusivo
#4-Não realizado
#5-Aguarda resultado

Comentário do Leonardo: Faltou classificar a variável sexo que eu fiz no começo da classificaçao e também das variáveis Fator de Risco e do target que eu fiz logo abaixo, pois acredito que ela precisa estar no formato 0 e 1. Neste caso peço que dêem uma atenção nesse caso antes de rodarem os modelos. Eu rodei a árvore e deu uma acurácia de 0,77 mas fiquei com uma pulga atrás da orelha ao analisar os resultados da classificação.


In [ ]:
df_sp_1['FATOR_RISC'].replace({"S": 1, "N": 0}, inplace= True)
print(df_sp_1['FATOR_RISC'].unique())
df_sp_1['FATOR_RISC'].isnull().sum()/len(df_sp_1['FATOR_RISC'])
#NOVO DICIONÁRIO: 
#0-Não
#1-Sim


In [ ]:
print(df_sp_1['EVOLUCAO'].nunique())
print(df_sp_1['EVOLUCAO'].unique())
df_sp_1['EVOLUCAO'].isnull().sum()/len(df_sp_1['EVOLUCAO'])
#1-Cura
#2-Óbito

Comentário do Leonardo: Precisei re-classificar a variável target, pois acredito que ela precisa ser uma dummy 0 e 1.

In [ ]:
df_sp_1['EVOLUCAO'].replace({2.0: 99, 1.0: 100}, inplace= True)
print(df_sp_1['EVOLUCAO'].unique())
df_sp_1['EVOLUCAO'].isnull().sum()/len(df_sp_1['EVOLUCAO'])
#NOVO DICIONÁRIO: 
#1-Óbito
#0-Cura


In [ ]:
df_sp_1['EVOLUCAO'].replace({99: 1, 100: 0}, inplace= True)
print(df_sp_1['EVOLUCAO'].unique())
df_sp_1['EVOLUCAO'].isnull().sum()/len(df_sp_1['EVOLUCAO'])
#NOVO DICIONÁRIO: 
#1-Óbito
#0-Cura


- ORDENAÇÃO DAS COLUNAS 

In [ ]:
Col= ['DT_NOTIFIC', 'IDADE_ANOS','CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N','HISTO_VGM', 
      'SURTO_SG', 'NOSOCOMIAL', 'AVE_SUINO','OUT_ANIM', 'FEBRE','TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO',
      'DIARREIA', 'VOMITO','DOR_ABD',  'FADIGA', 'PERD_OLFT', 'PERD_PALA', 'OUTRO_SIN', 'FATOR_RISC', 'PUERPERA', 
      'CARDIOPATI', 'HEMATOLOGI', 'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 
      'RENAL', 'OBESIDADE', 'OUT_MORBI', 'VACINA', 'ANTIVIRAL', 'CO_MU_INTE', 'UTI', 'DT_ENTUTI', 'DT_SAIDUTI', 
      'SUPORT_VEN', 'RAIOX_RES','TOMO_RES','PCR_RESUL', 'RES_IGG', 'RES_IGM', 'RES_IGA', 'EVOLUCAO', 'DT_EVOLUCA', 
      'DT_ENCERRA']    
     
df_sp_2 = df_sp_1.reindex(columns=Col)
df_sp_2.head()

### Comparações:


In [ ]:
print(df_brasil_AV.shape)
df_brasil_AV.isnull().sum()/len(df_brasil_AV)

In [ ]:
print(df_sp.shape)
df_sp.isnull().sum()/len(df_sp)

In [ ]:
print(df_sp_2.shape)
df_sp_2.isnull().sum()/len(df_sp_2)

Histograma

In [ ]:
plt.hist(df_sp_2["IDADE_ANOS"])

Aplicação do teste de Shapiro-Wilks

In [ ]:
stat, p = shapiro(df_sp_2["IDADE_ANOS"])
print(p)

Comentário do Leonardo: Como tivemos um p-valor = 0,0, a variável Idade_Anos não possui um comportamento normal, porém optei por não aplicar o normatização pelo stardard_scale devido a falta de tempo

### Exportação do CSV limpo.
 


In [ ]:
df_sp_2.to_csv(r'internados_sp_limpo.csv',index = False)

Split da Base

Comentário do Leonardo: Para compor a base a ser utilizada no modelo, eu retirei as datas e a variável vacina, pois eu acredito que não faça sentido prático esta variável estar na base no período anterior a aplicação da vacina

In [ ]:
X = df_sp_2.drop(columns=["EVOLUCAO", "DT_NOTIFIC", "OUT_ANIM", "VACINA", "DT_ENTUTI", "DT_SAIDUTI", "DT_EVOLUCA", 
                          "DT_ENCERRA"])
y = df_sp_2['EVOLUCAO']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
clf = DecisionTreeClassifier(criterion='gini', min_samples_leaf=30000)
clf.fit(Xtrain,ytrain)

In [ ]:
previsoes = clf.predict(Xtest)
previsoes

Aplicação da Árvore de Decisão

In [ ]:
accuracy_score(ytest, previsoes)

Comentário do Leonardo: Não sei o que houve para não ter conseguido rodar a matriz de confusão, tentei instalar o yellowbrick no Jupyter, mas não está reconhecendo a biblioteca 

In [ ]:
cm = ConfusionMatrix(clf)
cm.fit(Xtrain, ytrain)
cm.score(Xtest, ytest)

Saberiam me dizer o motivo de não ter os indicadores relacionados ao target=1? Eu tentei gerar a Árvore de Decisão, mas não consegui também

In [ ]:
print(classification_report(ytest, previsoes))